# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [1]:
import os
import sys
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [2]:
%store -r timestamp USE_FROZEN
if USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
print(f"Timestamp: {timestamp}")
outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Timestamp: 2024-07-14


### Calculation of proportion

In [3]:
# take only the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]
print(noc_cols)

['UnInfected_Cytosol_1', 'UnInfected_Cytosol_2', 'UnInfected_Cytosol_3', 'UnInfected_Nuclear_1', 'UnInfected_Nuclear_2', 'UnInfected_Nuclear_3', 'UnInfected_Organelle_1', 'UnInfected_Organelle_2', 'UnInfected_Organelle_3']


In [4]:
# just select the noc_cols + Protein IDs for merge purpose
noc_table = noc_table[['Protein IDs'] + noc_cols].copy()

In [5]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(proteingroup=None, file_designated=True,
    info_cols=['Protein IDs'], sample_cols=noc_cols)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# transform the intensity to linear space
#noc_process.transform_intensities(func=np.exp2)

# group replicates
noc_process.group_replicates(reg_exp=r'(.*)_\d+$')

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,UnInfected_Cytosol,UnInfected_Organelle,UnInfected_Nuclear,Protein IDs
2,75060000.0,76811000.0,166520000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,245940000.0,194700000.0,238020000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,4330700.0,51652000.0,22978000.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,3272600.0,0.0,0.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,9258400.0,77290000.0,24210000.0,Q99805;A0A024QYR8;B3KSG9


In [6]:
# we then take the sum of each column 
noc_medians['noc_sum_uninf'] = noc_medians['UnInfected_Cytosol'] + noc_medians['UnInfected_Organelle'] + noc_medians['UnInfected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_uninf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol'] = noc_medians['UnInfected_Cytosol'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_organelle'] = noc_medians['UnInfected_Organelle'] / noc_medians['noc_sum_uninf']
noc_proportion['NOC_nuclear'] = noc_medians['UnInfected_Nuclear'] / noc_medians['noc_sum_uninf']


### Merge NOC proportion with IP enrichment

In [7]:
# merge with the enrichment table
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [8]:
# finally, merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on="Protein IDs", how="inner")

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [9]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table_NOC_prop.csv"
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [10]:
# enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8536                                             X5D7P8   
8537                                             X5D8X9   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8539                                             X5DQZ7   
8540                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8536                                             X5D7P8            STK39   
8537                                             X5D8X9          CNTNAP2   
8538  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8539                                             X5DQZ7             GPX1   
8540                                             X6RHX1            SCYL3   

        sample                                                              \
      13-RAB7A   12-RTN4  13-RAB14 12-SEC61B 12-TOMM20 12-PNPLA2   12-ACTB   
0     0.926145 -0.089457  0.277650 -0.486730 -0.736898  0.594597 -0.179236   
1    -1.335100 -2.264500 -0.898950 -0.623300 -2.246100  0.710800  0.860750   
2     5.032826  4.511831  4.840405  4.783531 -0.436809 -4.775289 -4.635126   
3     1.142212  0.064925 -0.376243  0.447371 -0.030686 -0.429486  0.329705   
4     8.977542  6.673675  7.638707  6.847375 -2.183857 -7.775508 -5.963207   
...        ...       ...       ...       ...       ...       ...       ...   
8536  0.660475 -1.063275  0.342780 -0.251286  1.136048  0.383908 -0.365514   
8537  0.064876 -1.042621  0.963853 -0.715858 -0.678593 -0.153141 -0.061779   
8538 -0.581294  0.320812 -0.496372 -0.355636  0.627502  0.130662  0.126567   
8539 -0.664371 -0.571915 -0.755153 -0.186079  1.974409 -0.646730  0.180603   
8540 -0.675243 -0.557370  0.376539  0.393936 -0.696068 -0.282818  0.589653   

      ...                                                              \
      ... 09-ATG101     09-WT 10-TOMM20  09-PSMB7  10-EXOC2   09-EDC4   
0     ...  2.692774  2.090668 -0.589472 -1.432714 -0.492929  2.342249   
1     ... -0.728450  1.836900 -2.969400 -0.327100  1.017250  1.614550   
2     ...  2.393000  0.315600 -2.685600 -4.585382 -4.497900 -0.149000   
3     ... -0.381822  0.597445  0.105317 -0.704127 -1.008054 -0.449829   
4     ... -0.412350 -0.744500 -2.382900 -1.587400  0.033350 -1.223000   
...   ...       ...       ...       ...       ...       ...       ...   
8536  ...  1.261714 -0.194651  0.403811 -0.676353 -0.282873  0.652653   
8537  ...  1.109283 -1.293554 -0.427328 -0.901160 -1.289691  0.822587   
8538  ... -0.775370 -0.713403 -0.122160 -0.392848  0.632196  0.264631   
8539  ... -0.635694  0.105294  3.143259  0.161044  0.166296 -0.071422   
8540  ...  0.164947 -0.721191  0.008961  0.054858  0.748928 -0.688106   

                                                      
      10-VPS35 NOC_cytosol NOC_organelle NOC_nuclear  
0    -0.812759    0.235748      0.241247    0.523005  
1    -0.959200    0.362391      0.286889    0.350721  
2     0.601500    0.054846      0.654148    0.291006  
3    -0.567424    1.000000      0.000000    0.0000